## Workshop 06
1. หากต้องการกำหนดวันเริ่มต้นของการพิจารณาว่าไม่มีการใช้งานถือว่าย้ายค่ายเป็นวันใหม่ต้องกำหนดอย่างไร
2. หากต้องการกำหนดค่าการย้ายค่าย (y) เป็น 0.5  สำหรับลูกค้าที่หยุดใช้งานไปแล้วกลับมาใช้ใหม่และสถานะไม่มีการค้างชำระ จะต้องทำอย่างไร (คนที่ย้ายไปเลยให้เป็น 1 เหมือนเดิม)

(ตอบแต่ละข้อโดยการกำหนดที่ Code แล้วส่งเฉพาะ Code ส่วนที่เปลี่ยน)

In [ ]:
# กำหนดวันใหม่ใน function นี้โดยวันเริ่มต้นของการพิจารณาจะคำนวณจาก prediction_date - pd.DateOffset(days=lead_time_days)
# ซึ่งหมายความว่าวันเริ่มต้นจะเปลี่ยนแปลงตามตัวแปร lead_time_days โดยการเปลี่ยนแปลงค่านี้ต้องอาศัยการพูดคุยกับผู้เชี่ยวชาญด้วย

def calculate_churn_for_month(customer_info, raw_transactions, loans, prediction_date, lead_time_days=28):
    """
    Function to calculate churn labels for a specific month.
    A customer is labeled as churned if they have no transactions within the lead time period
    or if their loan status is 'Default'.
    """
    """
    ฟังก์ชันสำหรับคำนวณการสูญเสียลูกค้าในเดือนที่ระบุ
    ลูกค้าจะถูกจัดว่าสูญเสียในกรณีใดกรณีหนึ่งต่อไปนี้:
        1. ไม่มีการทำธุรกรรมใดๆ ในช่วงเวลาที่กำหนด
        2. สถานะเงินกู้ของลูกค้าอยู่ในสถานะ 'ผิดนัดชำระ'
    """
    # Define the window of time for which churn is being predicted
    start_date = prediction_date - pd.DateOffset(days=lead_time_days)
    end_date = prediction_date

    # Find recent transactions within the specified window
    recent_transactions = raw_transactions[
        (raw_transactions['TransactionDate'] >= start_date) &
        (raw_transactions['TransactionDate'] < end_date)
    ].groupby('CustomerID').size()

    # Determine churned customers: no transactions or loan in default
    churned_customers = customer_info['CustomerID'].apply(lambda x: 1 if (x not in recent_transactions.index) else 0)
    churned_customers = churned_customers | (loans.set_index('CustomerID')['LoanStatus'] == 'Default').astype(int)

    return churned_customers

In [ ]:
# จากโจทย์กำหนดให้ว่าหากย้ายค่ายแล้วกลับมาใช้งานให้เป็น 0.5 ซึ่งหมายความว่าหากค่าในคอลัมน์ที่เริ่มต้นด้วย '2023-'
# มีผลรวมมากกว่า 0 แต่ไม่ถึง 11 นั่นหมายความว่าลูกค้ามีการย้ายค่ายแล้วกลับมาใช้งานซึ่ง y=0.5
# แต่หากผลรวมแล้วได้ 11 หมายความว่าลูกค้าย้ายค่ายไปแล้ว y=1
# และหากผลผลรวมเป็น 0 หมายความว่าลูกค้าไม่ย้ายค่าย y=0

y = np.where(
    (datamart[[col for col in datamart.columns if col.startswith('2023-')]].sum(axis=1) > 0) &
    (datamart[[col for col in datamart.columns if col.startswith('2023-')]].sum(axis=1) < 11),
    0.5,
    np.where(
        (datamart[[col for col in datamart.columns if col.startswith('2023-')]].sum(axis=1) == 11),
        1,
        0
    )
)
y